In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re

In [3]:
# llegamos a las actividades en malaga
driver = webdriver.Chrome()             
url = 'https://www.civitatis.com/es/'
driver.get(url)
driver.maximize_window()  
driver.find_element('css selector','#didomi-notice-agree-button > span').click()
driver.find_element('css selector','#home-search-text').click()
sleep(5)
driver.find_element('css selector','#home-search-text').send_keys('malaga',Keys.TAB)
driver.find_element('css selector','#page-header__search-button').click()

WebDriverException: Message: unknown error: failed to change window state to 'maximized', current state is 'minimized'
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00007FF6E176B095+29557]
	(No symbol) [0x00007FF6E16DFA50]
	(No symbol) [0x00007FF6E159B56A]
	(No symbol) [0x00007FF6E1573FCE]
	(No symbol) [0x00007FF6E15724B5]
	(No symbol) [0x00007FF6E15708D9]
	(No symbol) [0x00007FF6E16506D1]
	(No symbol) [0x00007FF6E161718A]
	(No symbol) [0x00007FF6E163851C]
	(No symbol) [0x00007FF6E1616F33]
	(No symbol) [0x00007FF6E15E116F]
	(No symbol) [0x00007FF6E15E22D1]
	GetHandleVerifier [0x00007FF6E1A9C96D+3378253]
	GetHandleVerifier [0x00007FF6E1AE8497+3688311]
	GetHandleVerifier [0x00007FF6E1ADD1CB+3642539]
	GetHandleVerifier [0x00007FF6E182A6B6+813462]
	(No symbol) [0x00007FF6E16EAB5F]
	(No symbol) [0x00007FF6E16E6B74]
	(No symbol) [0x00007FF6E16E6D10]
	(No symbol) [0x00007FF6E16D5C1F]
	BaseThreadInitThunk [0x00007FFE0ECA7374+20]
	RtlUserThreadStart [0x00007FFE1045CC

In [394]:
pagina = 1
ciudad = 'malaga'
url = f'https://www.civitatis.com/es/{ciudad}/{pagina}'

In [398]:

def carga_datos():
    """Cargar datos de actividades desde una página web.

    Realiza una solicitud HTTP a la URL especificada y, si la solicitud es exitosa,
    analiza el contenido HTML para extraer información sobre actividades. La función
    busca los títulos, descripciones y precios de las actividades disponibles, y 
    devuelve esta información en un DataFrame de pandas.

    Returns:
        pd.DataFrame: Un DataFrame que contiene tres columnas: 'Actividad', 
        'Descripción' y 'Precio'. Si la solicitud falla o no se encuentran 
        productos, se devuelve un DataFrame vacío.
    """
    res = requests.get(url)
    if res.status_code == 200:
        print("Solicitud exitosa")
    else:
        print("Error en la solicitud")
        return pd.DataFrame()  

    sopa_actividades = BeautifulSoup(res.content, "html.parser")
    lista_productos = sopa_actividades.findAll("div", {'class': 'o-search-list__item'})
    
    titulos = []
    descripciones = []
    precios = []
    for producto in lista_productos:
    
        titulo_elemento = producto.find('h2')
        descripcion_elemento = producto.find("div", {'class': 'comfort-card__text l-list-card__text'})
        precio_elemento = producto.find("span", {'class': 'comfort-card__price__text'})
        
        if titulo_elemento and descripcion_elemento and precio_elemento:
            titulo = titulo_elemento.get_text(strip=True)
            descripcion = descripcion_elemento.get_text(strip=True)
            precio = precio_elemento.get_text(strip=True)
            
            
            titulos.append(titulo)
            descripciones.append(descripcion)
            precios.append(precio)
      
  
    df = pd.DataFrame({
        'Actividad': titulos,
        'Descripción': descripciones,
        'Precio': precios,
    })

    return df


df_actividades = carga_datos()

df_actividades

Solicitud exitosa


,Actividad,Descripción,Precio
0,Excursión al Caminito del Rey,En estaexcursión al Caminito del Reyviajaremos...,59€
1,Free tour por Málaga,"La Catedral, la casa de Picasso… Estos son alg...",¡Gratis!
2,Tour de Málaga al completo con entradas,En estetour por Málaga al completo con entrada...,36€
3,Excursión a Ronda y Setenil,"Si os gusta la historia, no podéis perderos es...",33€
4,Visita guiada por la alcazaba y el teatro romano,Lavisita guiada por la alcazaba y el teatro ro...,15€
5,Traslados en Málaga,"Servicioprivado, puerta a puerta y al mejor pr...","30,50€"
6,Excursión a Marbella y Mijas,En estaexcursión a Marbella y Mijasdescubrirem...,36€
7,"Excursión a Nerja, Frigiliana y El Acebuchal",Si quieresvisitar Nerja y conocer dos de los p...,75€
8,Excursión a Gibraltar,En estaexcursión a Gibraltarharemos untour pan...,79€
9,Free tour por la Alcazaba y el Teatro Romano,Sumérgete en la Historia visitando unantiguo t...,¡Gratis!


In [401]:
df_actividades.to_csv('Datos/actividades_malaga.csv')

In [4]:
df_actividades = pd.read_csv('../Datos/actividades_malaga.csv')
df_actividades.head()


,Unnamed: 0,Actividad,Descripción,Precio
0,0,Excursión al Caminito del Rey,En estaexcursión al Caminito del Reyviajaremos...,59€
1,1,Free tour por Málaga,"La Catedral, la casa de Picasso… Estos son alg...",¡Gratis!
2,2,Tour de Málaga al completo con entradas,En estetour por Málaga al completo con entrada...,36€
3,3,Excursión a Ronda y Setenil,"Si os gusta la historia, no podéis perderos es...",33€
4,4,Visita guiada por la alcazaba y el teatro romano,Lavisita guiada por la alcazaba y el teatro ro...,15€


In [5]:
df_actividades.dtypes

Unnamed: 0      int64
Actividad      object
Descripción    object
Precio         object
dtype: object

In [6]:
## pasamos a float el precio
df_actividades['Precio'] = df_actividades['Precio'].replace('¡Gratis!', '0')
df_actividades.head(5)

,Unnamed: 0,Actividad,Descripción,Precio
0,0,Excursión al Caminito del Rey,En estaexcursión al Caminito del Reyviajaremos...,59€
1,1,Free tour por Málaga,"La Catedral, la casa de Picasso… Estos son alg...",0
2,2,Tour de Málaga al completo con entradas,En estetour por Málaga al completo con entrada...,36€
3,3,Excursión a Ronda y Setenil,"Si os gusta la historia, no podéis perderos es...",33€
4,4,Visita guiada por la alcazaba y el teatro romano,Lavisita guiada por la alcazaba y el teatro ro...,15€


In [10]:
df_actividades['Precio'] = df_actividades['Precio'].str.replace('€', '')
df_actividades['Precio'] = df_actividades['Precio'].str.replace(',', '.')
df_actividades.head(2)

,Unnamed: 0,Actividad,Descripción,Precio
0,0,Excursión al Caminito del Rey,En estaexcursión al Caminito del Reyviajaremos...,59
1,1,Free tour por Málaga,"La Catedral, la casa de Picasso… Estos son alg...",0


In [11]:
df_actividades['Precio'] = df_actividades['Precio'].astype(float)

In [12]:
df_actividades.to_csv('../Datos/actividades_malaga.csv')